In [2]:
from help_functions import download_and_unzip
import pandas as pd

download_and_unzip('https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip', 'getting_started.zip')

Beginning file download...


In [34]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

#shuffle data
train_df_shuffled = train_data.sample(frac=1, random_state=42)

In [35]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [25]:
train_data['text'][4]

'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '

In [31]:
len(train_data[train_data.target == 1])

3271

In [52]:
import random
random_index = random.randint(0,len(train_df_shuffled)-5)

for row in train_df_shuffled[random_index:random_index+5][['text', 'target']].itertuples():
    _,text,target = row
    print(f'Target: {target}', '(real disaster)' if target == 1 else '(not real disaster)')
    print(f'Text:\n{text}\n---')


Target: 0 (not real disaster)
Text:
@TelegraphWorld lets hope it's a upper class white mass murderer....''' Mmmm
---
Target: 0 (not real disaster)
Text:
@okgabby_ damn suh. don't let that ruin your year bruh. this our year. better start carpooling like we did back in the day
---
Target: 0 (not real disaster)
Text:
@mfalcon21 go look. Just blew it up w atomic bomb.
---
Target: 0 (not real disaster)
Text:
@suelinflower there is no words to describe the physical painthey ripped you apart while you screamed for dear lifeits like been engulfed
---
Target: 1 (real disaster)
Text:
@UN No more #GujaratRiot &amp; #MumbaiRiot92-93 which devastated 1000&amp;1000 Indianperpetrated by #Modi &amp; #ChawalChorbjp @UN_Women  @UNNewsTeam
---


In [57]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_list(),
                                                                            train_df_shuffled['target'].to_list(), 
                                                                            test_size=0.1, 
                                                                            random_state=42)

In [60]:
train_sentences[:2], train_labels[:2]

(['@mogacola @zamtriossu i screamed after hitting tweet',
  'Imagine getting flattened by Kurt Zouma'],
 [0, 0])

In [67]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=10000, 
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=15,
                                    pad_to_max_tokens=True)

In [70]:
text_vectorizer.adapt(train_sentences)
sentence = 'there is a flood in my street.'
text_vectorizer([sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [72]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5], words_in_vocab[-5:]

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [73]:
#Creating an Embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=10000,
                                    output_dim=128,
                                    input_length=15)

In [75]:
import random
random_sentence = random.choice(train_sentences)
print(random_sentence)

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed


Monkeys Abused by Notorious Laboratory Dealer | A PETA Eyewitness Invest... https://t.co/QGqlpmRfJd via @YouTube


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03773749, -0.03120768, -0.00869508, ..., -0.03675335,
         -0.03229596, -0.01112098],
        [-0.01731961, -0.0207919 ,  0.02772314, ..., -0.0224804 ,
          0.04890342, -0.00418376],
        [ 0.00633465,  0.00482439,  0.02753774, ...,  0.01415584,
         -0.04650433, -0.01266574],
        ...,
        [ 0.03543358, -0.02374737, -0.00389288, ..., -0.01089724,
          0.00814679, -0.02317633],
        [ 0.04319842,  0.00416474, -0.01044507, ...,  0.00500754,
         -0.00304751, -0.03148011],
        [ 0.04319842,  0.00416474, -0.01044507, ...,  0.00500754,
         -0.00304751, -0.03148011]]], dtype=float32)>

In [78]:
sample_embed[0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.037737492>